# Tarefa 4: Traga seu próprio contêiner para o SageMaker Studio

Neste notebook, você vai criar uma imagem do Docker e um contêiner de processamento. Você vai usar uma classe **ScriptProcessor** do SDK Python do Amazon SageMaker para executar um script scikit-learn de pré-processamento no contêiner. Depois você vai validar os resultados do processamento de dados salvos no Amazon Simple Storage Service (Amazon S3).

## Tarefa 4.1: Configuração do ambiente

Instale as bibliotecas e dependências necessárias.

Você vai definir um bucket do Amazon S3 para armazenar as saídas do trabalho de processamento e também fazer com que o perfil de execução execute o trabalho de processamento do SageMaker.

In [ ]:
%%capture

# add csv visualization and image-build packages
%pip install --upgrade sagemaker-datawrangler
%pip install sagemaker-studio-image-build 
%pip install --upgrade pandas

In [ ]:
#install-dependencies
import logging
import boto3
import sagemaker
import pandas as pd
import numpy as np
import sagemaker_datawrangler
from sagemaker.s3 import S3Downloader

sagemaker_logger = logging.getLogger("sagemaker")
sagemaker_logger.setLevel(logging.INFO)
sagemaker_logger.addHandler(logging.StreamHandler())

sagemaker_session = sagemaker.Session()

#Execution role to run the SageMaker Processing job
role = sagemaker.get_execution_role()
print("SageMaker Execution Role: ", role)

#S3 bucket to read the SKLearn processing script and writing processing job outputs
s3 = boto3.resource('s3')
for buckets in s3.buckets.all():
    if 'databucket' in buckets.name:
        bucket = buckets.name
print("Bucket: ", bucket)

prefix = 'scripts/data'
S3Downloader.download(s3_uri=f"s3://{bucket}/{prefix}/abalone_data.csv", local_path= 'data/')

## Tarefa 4.2: Criar um contêiner de processamento

Defina e crie um contêiner do scikit-learn usando o Dockerfile.

### Tarefa 4.2.1: Criar um Dockerfile

Crie um diretório do Docker e adicione o Dockerfile que cria o contêiner de processamento. Por estar criando um contêiner scikit-learn, você vai instalar o pandas e o scikit-learn.

In [ ]:
%mkdir docker

In [ ]:
%%writefile docker/Dockerfile
FROM public.ecr.aws/docker/library/python:3.10-slim-bullseye

RUN pip3 install pandas scikit-learn
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3"]

### Tarefa 4.2.2: Criar a imagem de contêiner

Crie uma imagem de contêiner personalizada usando a interface de linha de comando de criação de imagem do Amazon SageMaker Studio.

Usando a CLI de criação de imagem do Amazon SageMaker Studio, você pode criar imagens do Docker compatíveis com o Amazon SageMaker diretamente nos ambientes do SageMaker Studio. A CLI de criação de imagem poupa tempo e aumenta a segurança porque abstrai a criação do ambiente de criação e exige menos permissões.

Navegue até o diretório que contém o Dockerfile e execute o comando de criação sm-docker. Esse comando registra em log automaticamente a saída da criação e retorna o **URI da imagem** do Docker. Essa etapa leva de 2 a 5 minutos.

In [ ]:
%%sh

rm /usr/lib/x86_64-linux-gnu/libstdc++.so.6

cp /opt/conda/lib/libstdc++.so.6 /usr/lib/x86_64-linux-gnu/libstdc++.so.6

cd docker

sm-docker build .

Quando a execução da célula terminar, vai aparecer um URI de imagem semelhante a este: *012345678910.dkr.ecr.us-east-1.amazonaws.com/sagemaker-studio-d-vcbyjgmmjzzy:data-scientist-test-user*.

1. Copie o **URI da imagem** e cole-o em um editor de texto da sua escolha. 

Use esse **URI da imagem** para criar uma classe **ScriptProcessor**.

## Tarefa 4.3: Executar o trabalho de processamento do SageMaker

A AnyCompany Consulting está trabalhando em um projeto com um grupo ambiental sobre a idade de abalones. Abalones são um tipo de molusco ou lesma marinha. Eles querem prever a idade de espécimes vivos em vez de cortar as conchas dos animais para determinar a idade.

O conjunto de dados de abalones representa uma população de mais de 4.000 espécimes. O conjunto de dados contém colunas referentes a sexo, comprimento, diâmetro, altura, peso total, peso sem a concha, peso das vísceras, peso da concha e anéis.

Execute um trabalho de processamento no conjunto de dados de abalones.

In [ ]:
#import-data
shape=pd.read_csv("data/abalone_data.csv", header=0)
shape.sample(5)

Depois use a classe ScriptProcessor do SageMaker para definir e executar um script de processamento como um trabalho de processamento. Consulte [SageMaker ScriptProcessor](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor) para saber mais sobre essa classe.

Para criar a classe ScriptProcessor, configure os seguintes parâmetros:
- **base_job_name**: prefixo do nome do trabalho de processamento
- **command**: comando a ser executado, além dos sinalizadores da linha de comando
- **image_uri**: URI da imagem do Docker a ser usado para os trabalhos de processamento
- **role**: função de execução do SageMaker
- **instance_count**: número de instâncias para executar o trabalho de processamento
- **instance_type**: tipo de instância do Amazon Elastic Compute Cloud (Amazon EC2) usado para o trabalho de processamento

1. No código a seguir, substitua **REPLACE_IMAGE_URI** pelo URI do editor de texto.

In [ ]:
#sagemaker-script-processor
from sagemaker.processing import ScriptProcessor

# create a ScriptProcessor
script_processor = ScriptProcessor(
    base_job_name="own-processing-container",
    command=["python3"],
    image_uri="REPLACE_IMAGE_URI",
    role=role,
    instance_count=1,
    instance_type="ml.m5.xlarge",
)

Depois use o método ScriptProcessor.run() para executar o script **sklearn_preprocessing.py** como um trabalho de processamento. Consulte [ScriptProcessor.run()](https://sagemaker.readthedocs.io/en/stable/api/training/processing.html#sagemaker.processing.ScriptProcessor.run) para saber mais sobre esse método.

Para executar o trabalho de processamento, configure os seguintes parâmetros:
- **code**: caminho do script de pré-processamento 
- **inputs**: caminho dos dados de entrada do script de pré-processamento (local de entrada do Amazon S3)
- **outputs**: caminho da saída do script de pré-processamento (local de saída do Amazon S3)
- **arguments**: argumentos de linha de comando para o script de pré-processamento (como a taxa de divisão do teste de treinamento)

O trabalho de processamento exige cerca de quatro a cinco minutos para ser concluído.

In [ ]:
#processing-job
import os
from sagemaker.processing import ProcessingInput, ProcessingOutput

# Amazon S3 path prefix
input_raw_data_prefix = "scripts/data"
output_preprocessed_data_prefix = "scripts/data/output"
scripts_prefix = "scripts/smstudiofiles"
logs_prefix = "logs"

# Run the processing job
script_processor.run(
    code="s3://" + os.path.join(bucket, scripts_prefix, "sklearn_preprocessing.py"),
    inputs=[ProcessingInput(source="s3://" + os.path.join(bucket, input_raw_data_prefix, "abalone_data.csv"),
                            destination="/opt/ml/processing/input")],
    outputs=[
        ProcessingOutput(output_name="train_data", 
                        source="/opt/ml/processing/train",
                        destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "train")),
        ProcessingOutput(output_name="test_data", 
                        source="/opt/ml/processing/test",
                        destination="s3://" + os.path.join(bucket, output_preprocessed_data_prefix, "test")),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
)
script_processor_job_description = script_processor.jobs[-1].describe()
print(script_processor_job_description)

## Tarefa 4.4: validar os resultados do processamento de dados

Valide a saída do trabalho de processamento que você executou consultando as cinco primeiras linhas dos conjuntos de dados de saída de treinamento e teste.

In [ ]:
#view-train-dataset
print("Top 5 rows from s3://{}/{}/train/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/train/train_features.csv - | head -n5

In [ ]:
#view-validation-dataset
print("Top 5 rows from s3://{}/{}/validation/".format(bucket, output_preprocessed_data_prefix))
!aws s3 cp --quiet s3://$bucket/$output_preprocessed_data_prefix/test/test_features.csv - | head -n5

Quando a execução das células terminar, vão aparecer respostas semelhantes a estas:

```plain
Top 5 rows from s3://databucket-us-east-1-xxxxxxx/scripts/data/output/validation/
M,0.55,0.425,0.155,0.918,0.278,0.243,0.335
I,0.5,0.4,0.12,0.616,0.261,0.143,0.194
M,0.62,0.48,0.155,1.256,0.527,0.374,0.318
I,0.22,0.165,0.055,0.055,0.022,0.012,0.02
M,0.645,0.5,0.175,1.511,0.674,0.376,0.378
```

Os cabeçalhos das colunas do conjunto de dados de abalones são: sex (Infant, Male, Female), length, diameter, height, whole_weight, shucked_weight, viscera_weight, shell_weight e rings. A saída das pastas **train/** e **validation/** mostra os dados processados armazenados no bucket do S3.

Você criou um contêiner de processamento e usou o processamento do SageMaker para executar o trabalho de processamento nele.

### Limpeza

Você concluiu este notebook. Passe para a próxima parte do laboratório da seguinte forma:

- Feche este arquivo de notebook.
- Retorne à sessão do laboratório e continue com a **Conclusão**.